In [1]:
print "ready"

ready


In [4]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from collections import defaultdict
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
import pprint
import os
import re

In [5]:
path = 'E:/SUBJECTS/497 MATH S17 PSP--/Data/'
interpath= 'E:/SUBJECTS/497 MATH S17 PSP--/Data/intermediate/'

In [4]:
encounters=pd.read_pickle(path + '/encounters.pickle')

In [18]:
print encounters.shape
encounters.info()

(85829, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 85829 entries, 10335 to 47927
Data columns (total 5 columns):
﻿Person_ID       85829 non-null object
Person_Nbr       85829 non-null int64
Enc_ID           85829 non-null object
Enc_Nbr          85829 non-null int64
Enc_Timestamp    85829 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 3.9+ MB


There are 85829 unique Enc_Nbrs and 16509 unique Person_Nbrs in encounters table

In [25]:
print len(encounters.Enc_Nbr.unique())
len(encounters.Person_Nbr.unique())

85829


16509

In [5]:
all_enc = pd.read_pickle(path + '/all_encounter_data.pickle') 

In [26]:
print all_enc.shape
all_enc.info()

(82432, 41)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 82432 entries, 75857 to 79638
Data columns (total 41 columns):
Enc_ID            82432 non-null object
Enc_Nbr           82432 non-null int64
Enc_Date          82432 non-null datetime64[ns]
Person_ID         82432 non-null object
Person_Nbr        82432 non-null int64
Primary_Payer     82432 non-null object
Smoking_Status    59199 non-null object
BMI               22325 non-null object
BP                38219 non-null object
Glucose           30483 non-null object
A1C               12793 non-null float64
MR_OD_SPH         25476 non-null object
MR_OD_CYL         24693 non-null object
MR_OD_AXIS        19897 non-null object
MR_OD_DVA         24179 non-null object
MR_OD_NVA         18104 non-null object
MR_OS_SPH         25452 non-null object
MR_OS_CYL         24606 non-null object
MR_OS_AXIS        19417 non-null object
MR_OS_DVA         24068 non-null object
MR_OS_NVA         18035 non-null object
BB_OD_SPH         5851 non-n

There are 80946 unique Enc_Nbrs and 16087 unique Person_Nbrs in all_enc table

In [27]:
print len(all_enc.Enc_Nbr.unique())
len(all_enc.Person_Nbr.unique())

80946


16087

## Findings: 

There are 3397 Enc_nbrs in encounter table which do not have data recorded for them in the all_enc table
There are 422 Person_nbrs in encounter table which do not have data recorded for them in the all_enc table

Out of the 82432 Enc_Nbrs in the all_enc table 80946 are unique leaving 1486 duplicated Enc_Nbrs

In [28]:
len(all_enc['Enc_Nbr'])-len(all_enc['Enc_Nbr'].drop_duplicates())

1486

## IDEA: Create a table for each person with a single encounter data set where examination values are averaged

## Distribution and Outliers  

Separate BP into BP_Systolic and BP_Diastolic

In [73]:
pattern = re.compile("(?P<BP_Systolic>\d+)\s*\/\s*(?P<BP_Diastolic>\d+)")
all_enc = pd.merge(all_enc, all_enc["BP"].str.extract(pattern, expand=True),
                         left_index=True, right_index=True)

Fix the 87 values in Glucose column with x/y format

In [75]:
result=all_enc["Glucose"].str.extract(pattern, expand=True)
result.info()

In [77]:
all_enc=pd.merge(all_enc,result,left_index=True,right_index=True)
all_enc.head()

,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,Primary_Payer,Smoking_Status,BMI,BP,Glucose,...,CYCLO_OD_NVA,CYCLO_OS_SPH,CYCLO_OS_CYL,CYCLO_OS_AXIS,CYCLO_OS_DVA,CYCLO_OS_NVA,BP_Systolic_x,BP_Diastolic_x,BP_Systolic_y,BP_Diastolic_y
75857,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,Premier Vision Plan,Never smoker,NaN,NaN,130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75858,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,Premier Vision Plan,Never smoker,NaN,118/ 78,130,...,NaN,NaN,NaN,NaN,NaN,NaN,118,78,NaN,NaN
55347,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,Adv BCBS HMO,Former smoker,26.04,140 / 88,140,...,NaN,NaN,NaN,NaN,NaN,NaN,140,88,NaN,NaN
55348,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,Adv BCBS HMO,Former smoker,29.16,128 / 78,120,...,NaN,NaN,NaN,NaN,NaN,NaN,128,78,NaN,NaN
55349,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,Adv BCBS HMO,Former smoker,29.16,132 / 86,130,...,NaN,NaN,NaN,NaN,NaN,NaN,132,86,NaN,NaN


In [79]:
print all_enc.BP_Systolic_y.isnull().value_counts()
all_enc.BP_Diastolic_y.isnull().value_counts()

True     82345
False       87
Name: BP_Systolic_y, dtype: int64


True     82345
False       87
Name: BP_Diastolic_y, dtype: int64

In [80]:
all_enc.BP_Systolic_x=all_enc.BP_Systolic_x.convert_objects(convert_numeric=True)
all_enc.BP_Systolic_y=all_enc.BP_Systolic_y.convert_objects(convert_numeric=True)
BPS = pd.concat((all_enc.BP_Systolic_x, all_enc.BP_Systolic_y),axis=1).mean(axis=1)
all_enc.BP_Systolic_x=BPS

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [81]:
all_enc.BP_Diastolic_x=all_enc.BP_Diastolic_x.convert_objects(convert_numeric=True)
all_enc.BP_Diastolic_y=all_enc.BP_Diastolic_y.convert_objects(convert_numeric=True)
BPD = pd.concat((all_enc.BP_Diastolic_x, all_enc.BP_Diastolic_y),axis=1).mean(axis=1)
all_enc.BP_Diastolic_x=BPD

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [82]:
pattern1 = re.compile("(\d+)\s*\/\s*(\d+)")
all_enc.Glucose=all_enc.Glucose.replace(pattern1,"NaN")

Converting to int 

In [84]:
toconvert=['A1C','BMI','Glucose']
for i in toconvert:
    all_enc[i]=all_enc[i].convert_objects(convert_numeric=True)

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


Change NaN to zero

In [86]:
tochange=['A1C','BMI','Glucose','BP_Systolic_x','BP_Diastolic_x']
for i in tochange:
    all_enc[i]=np.nan_to_num(all_enc[i])

Create a new set for analysis 

In [88]:
columns=['Person_Nbr','Enc_Nbr','Enc_Date','Smoking_Status','Primary_Payer','A1C','BMI','Glucose','BP_Systolic_x','BP_Diastolic_x']

In [92]:
all_enc_1= pd.DataFrame(columns=columns)

In [94]:
for index,row in all_enc.iterrows():
    for i in columns:
           all_enc_1.loc[index,i]=row[i]

In [99]:
all_enc_1.head()

,Person_Nbr,Enc_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x
75857,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8,0,130,0,0
75858,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8,0,130,118,78
55347,89,12870648,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0,26.04,140,140,88
55348,89,4126172,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0,29.16,120,128,78
55349,89,9217142,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0,29.16,130,132,86


In [102]:
all_enc_1.to_csv('all_enc_1.txt')

In [101]:
all_enc_1.shape

(82432, 10)

Based on the visualizations in R and research the following outlier ranges have been considered 

In [11]:
all_enc_2=pd.read_csv(interpath+'all_enc_1.csv')

In [13]:
all_enc_2.head()

,Unnamed: 0,Person_Nbr,Enc_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x
0,75857,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8.0,0.00,130.0,0.0,0.0
1,75858,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8.0,0.00,130.0,118.0,78.0
2,55347,89,12870648,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0
3,55348,89,4126172,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0
4,55349,89,9217142,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0


In [15]:
outliers = {
            "A1C" : lambda x: x if 3.89 < x < 18 else 0,
            "BMI" : lambda x: x if 8 < x < 204 else 0,
            "Glucose" : lambda x: x if 50 < x < 2656 else 0,
            "BP_Systolic_x" : lambda x: x if 55 < x < 370 else 0,
            "BP_Diastolic_x" : lambda x: x if 30 < x < 360 else 0
        }
       

In [16]:
 for column in list(outliers):
            all_enc_2[column] = all_enc_2[column].map(outliers[column])

In [17]:
all_enc_2.to_csv('all_enc_2.csv')

Taking average for the values recorded for same encounter, i.e the 1486 duplicated Enc_Nbrs

In [18]:
dups= pd.DataFrame(columns=dict(all_enc_2))

In [19]:
all_enc_2["Duplicate_Exists"]=all_enc_2["Enc_Nbr"].duplicated(keep=False)

In [20]:
all_enc_2.Duplicate_Exists.sum()

2806

In [ ]:
for index,row in all_enc_2.iterrows():
    if row["Duplicate_Exists"]==True:
        dups.loc[index]=row

0
1
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
34
35
36
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
65
66
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non dup
non d

In [204]:
dups.shape

(2806, 11)

In [198]:
dups.head()

,Unnamed: 0,BP_Diastolic_x,Enc_Nbr,BMI,Smoking_Status,BP_Systolic_x,Enc_Date,A1C,Unnamed: 0.1,Primary_Payer,Person_Nbr,Glucose
0,0,0.0,123227.0,0.0,Never smoker,0.0,2014-12-18 14:00:00,8.0,75857.0,Premier Vision Plan,33.0,130.0
1,1,78.0,123227.0,0.0,Never smoker,118.0,2014-12-18 14:00:00,8.0,75858.0,Premier Vision Plan,33.0,130.0
2,2,106.0,13886332.0,0.0,Never smoker,180.0,2015-03-03 18:15:00,0.0,71871.0,Blue Shield Of Illinois PPO,535.0,0.0
3,3,55.0,13886332.0,0.0,Never smoker,124.0,2015-03-03 18:15:00,0.0,71872.0,Blue Shield Of Illinois PPO,535.0,0.0
4,4,55.0,13886332.0,24.6,Never smoker,124.0,2015-03-03 18:15:00,0.0,71873.0,Blue Shield Of Illinois PPO,535.0,0.0


In [205]:
dups.to_csv('dups.csv')

In [28]:
columns=['A1C','BMI','Glucose','BP_Systolic_x','BP_Diastolic_x']

In [29]:
columns1=['Enc_Nbr','Enc_Date','Person_Nbr']

In [30]:
columns2=['Primary_Payer','Smoking_Status']

In [206]:
a=dups.groupby("Enc_Nbr")[columns].mean()

In [207]:
a.shape

(1320, 5)

In [208]:
a.head()

,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x
Enc_Nbr,,,,,
10196.0,0.0,0.00,0.0,126.5,65.0
31148.0,0.0,31.87,0.0,142.0,82.0
44311.0,7.0,0.00,105.0,70.0,45.0
46178.0,0.0,0.00,282.0,153.5,95.0
69786.0,0.0,0.00,131.0,189.5,99.0


In [209]:
b=dups.drop(columns, axis=1).drop_duplicates("Enc_Nbr").set_index("Enc_Nbr")

In [210]:
b.shape

(1320, 5)

In [211]:
b.head()

,Unnamed: 0,Smoking_Status,Enc_Date,Primary_Payer,Person_Nbr
Enc_Nbr,,,,,
123227.0,0,Never smoker,2014-12-18 14:00:00,Premier Vision Plan,33.0
13886332.0,2,Never smoker,2015-03-03 18:15:00,Blue Shield Of Illinois PPO,535.0
5330366.0,5,Never smoker,2016-06-10 18:30:00,Medicare,777.0
4612609.0,7,Former smoker,2012-01-12 00:00:00,Medicare,1614.0
2586704.0,11,Never smoker,2012-05-01 06:00:00,Medicare,2556.0


In [215]:
del b["Unnamed: 0"]

In [216]:
b.head()

,Smoking_Status,Enc_Date,Primary_Payer,Person_Nbr
Enc_Nbr,,,,
123227.0,Never smoker,2014-12-18 14:00:00,Premier Vision Plan,33.0
13886332.0,Never smoker,2015-03-03 18:15:00,Blue Shield Of Illinois PPO,535.0
5330366.0,Never smoker,2016-06-10 18:30:00,Medicare,777.0
4612609.0,Former smoker,2012-01-12 00:00:00,Medicare,1614.0
2586704.0,Never smoker,2012-05-01 06:00:00,Medicare,2556.0


In [218]:
dups1=pd.merge(a,b,left_index=True,right_index=True)

In [249]:
dups

(1320, 9)

In [220]:
dups1.head()

,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,Smoking_Status,Enc_Date,Primary_Payer,Person_Nbr
Enc_Nbr,,,,,,,,,
123227.0,8.0,0.000,130.0,59.000000,39.0,Never smoker,2014-12-18 14:00:00,Premier Vision Plan,33.0
13886332.0,0.0,8.200,0.0,142.666667,72.0,Never smoker,2015-03-03 18:15:00,Blue Shield Of Illinois PPO,535.0
5330366.0,0.0,0.000,188.0,163.000000,95.0,Never smoker,2016-06-10 18:30:00,Medicare,777.0
4612609.0,3.3,14.445,36.0,55.000000,33.0,Former smoker,2012-01-12 00:00:00,Medicare,1614.0
2586704.0,0.0,19.080,0.0,0.000000,0.0,Never smoker,2012-05-01 06:00:00,Medicare,2556.0


In [222]:
dups1.to_csv('dups1.csv')

In [223]:
all_enc_2.shape

(82432, 12)

In [224]:
all_enc_2.head()

,Unnamed: 0,Person_Nbr,Enc_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,Duplicate_Exists
0,75857,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8.0,0.00,130.0,0.0,0.0,True
1,75858,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8.0,0.00,130.0,118.0,78.0,True
2,55347,89,12870648,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0,False
3,55348,89,4126172,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0,False
4,55349,89,9217142,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0,False


In [250]:
c=all_enc_2.drop_duplicates("Enc_Nbr",keep=False).set_index("Enc_Nbr")

In [251]:
c.shape

(79626, 11)

In [253]:
del c["Unnamed: 0"]

In [254]:
c.head()

,Person_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,Duplicate_Exists
Enc_Nbr,,,,,,,,,,
12870648,89,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0,False
4126172,89,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0,False
9217142,89,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0,False
14968450,89,2014-05-25 10:45:00,Former smoker,Adv BCBS HMO,0.0,26.38,130.0,78.0,62.0,False
14537805,89,2015-03-14 15:30:00,Former smoker,Adv BCBS HMO,0.0,0.00,0.0,125.0,73.0,False


In [230]:
for i,j in c.iterrows():
    if j["Duplicate_Exists"]==True:
        print j    

In [256]:
c=c.reset_index()
dups1=dups1.reset_index()

In [257]:
all_enc_3=pd.merge(c,dups1,how="outer")

In [258]:
all_enc_3.shape

(80946, 12)

In [259]:
all_enc_3.head()

,index,Enc_Nbr,Person_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,Duplicate_Exists
0,0.0,12870648.0,89.0,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0,False
1,1.0,4126172.0,89.0,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0,False
2,2.0,9217142.0,89.0,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0,False
3,3.0,14968450.0,89.0,2014-05-25 10:45:00,Former smoker,Adv BCBS HMO,0.0,26.38,130.0,78.0,62.0,False
4,4.0,14537805.0,89.0,2015-03-14 15:30:00,Former smoker,Adv BCBS HMO,0.0,0.00,0.0,125.0,73.0,False


In [260]:
all_enc_3.to_csv('all_enc_3.csv')

Adding ICD codes

In [262]:
icd_for_enc=pd.read_pickle(path+'ICD_for_Enc.pickle')

In [263]:
print icd_for_enc.shape
icd_for_enc.info()

(207255, 7)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 207255 entries, 194529 to 201888
Data columns (total 7 columns):
Person_ID            207255 non-null object
Person_Nbr           207255 non-null int64
Enc_ID               207255 non-null object
Enc_Nbr              207255 non-null int64
Enc_Timestamp        207255 non-null datetime64[ns]
Diagnosis_Code_ID    207255 non-null object
Description          207251 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 12.6+ MB


In [264]:
icd_for_enc.head()

,Person_ID,Person_Nbr,Enc_ID,Enc_Nbr,Enc_Timestamp,Diagnosis_Code_ID,Description
194529,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,367.4,Presbyopia
194530,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,373.00,Blepharitis unspecified
194531,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,250.00,Diabetes mellitus without mention of complicat...
136763,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,365.02,Anatomical narrow angle borderline glaucoma
136764,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,250.00,Diabetes Mellitus Type 2 Uncomplicated


In [265]:
len(icd_for_enc.Enc_Nbr.unique())

64217

In [266]:
icd_for_enc.Diagnosis_Code_ID.describe()

count     207255
unique      2139
top       250.00
freq       18575
Name: Diagnosis_Code_ID, dtype: object

In [267]:
cols=["Enc_Nbr","Person_Nbr","Diagnosis_Code_ID"]

In [268]:
icd=icd_for_enc[cols]

In [269]:
icd=icd.reset_index()

In [270]:
icd.head()

,index,Enc_Nbr,Person_Nbr,Diagnosis_Code_ID
0,194529,123227,33,367.4
1,194530,123227,33,373.00
2,194531,123227,33,250.00
3,136763,12870648,89,365.02
4,136764,12870648,89,250.00


In [ ]:
count=1
index=0
alld_in=0
alld=pd.DataFrame(columns=["Enc_Nbr","ICD","Num_Encs"])
while index<len(icd):
    cur=icd.loc[index,"Enc_Nbr"]
    alld.loc[alld_in,"Enc_Nbr"]=cur
    nex=icd.loc[index+1,"Enc_Nbr"]
    alld.loc[alld_in,"ICD"]=icd.loc[index,"Diagnosis_Code_ID"]
    loop_count=1
    while cur==nex:
        alld.loc[alld_in,"ICD"] = alld.loc[alld_in,"ICD"] + ","+ icd.loc[index+1,"Diagnosis_Code_ID"]
        index=index+1
        nex=icd.loc[index+1,"Enc_Nbr"]
        loop_count=loop_count+1
        alld.loc[alld_in,"Num_Encs"] = loop_count
    if alld.loc[alld_in,"Num_Encs"] != loop_count :
        alld.loc[alld_in,"Num_Encs"] = count  
    alld_in=alld_in+1
    index=index+1

In [273]:
alld.shape

(64217, 4)

In [275]:
alld.to_csv('alld.csv')

In [272]:
alld=pd.read_csv(interpath+'alld.csv')

In [279]:
alld.head()

,Enc_Nbr,ICD,Num_Encs
0,123227,"367.4,373.00,250.00",3.0
1,12870648,"365.02,250.00",2.0
2,4126172,"362.81,365.02,250.00",3.0
3,9217142,365.02,1.0
4,14968450,"250.00,365.02,365.23,365.23",4.0


In [280]:
all_enc_4=pd.merge(all_enc_3,alld, how='left',on='Enc_Nbr')

In [281]:
all_enc_4.shape

(80946, 14)

In [6]:
ale=pd.read_csv(interpath+'all_enc_4.csv')

C:\Users\mom1\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
ale.head()

,index,Enc_Nbr,Person_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,ICD,Num_Encs
0,0.0,12870648.0,89.0,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0,"365.02,250.00",2.0
1,1.0,4126172.0,89.0,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0,"362.81,365.02,250.00",3.0
2,2.0,9217142.0,89.0,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0,365.02,1.0
3,3.0,14968450.0,89.0,2014-05-25 10:45:00,Former smoker,Adv BCBS HMO,0.0,26.38,130.0,78.0,62.0,"250.00,365.02,365.23,365.23",4.0
4,4.0,14537805.0,89.0,2015-03-14 15:30:00,Former smoker,Adv BCBS HMO,0.0,0.00,0.0,125.0,73.0,365.06,1.0


In [8]:
del ale["Duplicate_Exists"]
del ale["Unnamed: 0"]

In [10]:
dups_per= pd.DataFrame(columns=dict(ale))
ale["Duplicate_Exists"]=ale["Person_Nbr"].duplicated(keep=False)
ale.Duplicate_Exists.sum()

76401

In [13]:
ale.shape

(80946, 14)

In [14]:
count=0
for index,row in ale.iterrows():
    count=count+1
    if row["Duplicate_Exists"]==True:
        dups_per.loc[index]=row

1000
5000
10000
15000
20000
25000
30000
32000
42000
50000
60000
70000
80000


In [16]:
dups_per.shape

(76401, 13)

In [ ]:
dups_per.head()

In [ ]:
dups_per.to_csv('dups_per.csv')

In [17]:
columns=['A1C','BMI','Glucose','BP_Systolic_x','BP_Diastolic_x']

In [18]:
a=dups_per.groupby("Person_Nbr")[columns].mean()

In [19]:
a.shape

(11542, 5)

In [ ]:
a.head()

In [20]:
b=dups_per.drop(columns, axis=1).drop_duplicates("Person_Nbr").set_index("Person_Nbr")
print b.shape
b.head()

(11542, 7)


,index,Enc_Nbr,Smoking_Status,Enc_Date,Num_Encs,ICD,Primary_Payer
Person_Nbr,,,,,,,
89.0,0.0,12870648.0,Former smoker,2012-10-16 03:45:00,2.0,"365.02,250.00",Adv BCBS HMO
146.0,9.0,9086558.0,Current every day smoker,2012-11-04 19:45:00,5.0,"366.19,374.87,250.00,250.00,366.19",Adv Blue Shield Of Illinois
196.0,14.0,14534672.0,Never smoker,2016-10-05 00:45:00,4.0,"H10.45,H35.40,H04.123,H52.4",EyeQuest Family Health Network
331.0,17.0,2071158.0,Never smoker,2015-12-01 08:15:00,2.0,"H52.4,E11.331",Davis Vision Plan
535.0,21.0,4174447.0,Never smoker,2014-05-16 17:30:00,4.0,"362.02,361.81,362.02,361.81",Blue Shield Of Illinois PPO


In [21]:
dups_per1=pd.merge(a,b,left_index=True,right_index=True)
print dups_per1.shape
dups_per1.head()

(11542, 12)


,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,index,Enc_Nbr,Smoking_Status,Enc_Date,Num_Encs,ICD,Primary_Payer
Person_Nbr,,,,,,,,,,,,
89.0,0.000000,12.304444,73.00,84.222222,53.111111,0.0,12870648.0,Former smoker,2012-10-16 03:45:00,2.0,"365.02,250.00",Adv BCBS HMO
146.0,0.000000,6.420000,0.00,50.000000,29.000000,9.0,9086558.0,Current every day smoker,2012-11-04 19:45:00,5.0,"366.19,374.87,250.00,250.00,366.19",Adv Blue Shield Of Illinois
196.0,0.000000,0.000000,39.00,62.500000,44.000000,14.0,14534672.0,Never smoker,2016-10-05 00:45:00,4.0,"H10.45,H35.40,H04.123,H52.4",EyeQuest Family Health Network
331.0,2.733333,0.000000,126.00,112.000000,67.000000,17.0,2071158.0,Never smoker,2015-12-01 08:15:00,2.0,"H52.4,E11.331",Davis Vision Plan
535.0,0.000000,1.863750,28.75,49.666667,29.250000,21.0,4174447.0,Never smoker,2014-05-16 17:30:00,4.0,"362.02,361.81,362.02,361.81",Blue Shield Of Illinois PPO


In [ ]:
dups_per1.to_csv('dups_per1.csv')

In [22]:
c=ale.drop_duplicates("Person_Nbr",keep=False).set_index("Person_Nbr")
print c.shape
c.head()

(4545, 13)


,index,Enc_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,ICD,Num_Encs,Duplicate_Exists
Person_Nbr,,,,,,,,,,,,,
327.0,16.0,14361139.0,2011-12-04 11:00:00,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,"367.4,365.01,365.01,367.4,367.4",5.0,False
335.0,20.0,11783809.0,2013-10-16 17:45:00,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,"250.00,250.00",2.0,False
1515.0,132.0,6074701.0,2012-10-01 15:30:00,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,"250.00,373.00,367.4,250.00,373.00",5.0,False
1519.0,133.0,8583237.0,2014-07-01 17:45:00,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,"367.0,367.4,362.05,362.57,362.57,362.05,367.4",7.0,False
1628.0,156.0,9342972.0,2016-04-26 17:00:00,Never smoker,Medicaid HFS,0.0,0.00,0.0,135.0,71.0,"H04.123,E11.9,H52.223",3.0,False


In [23]:
c=c.reset_index()
dups_per1=dups_per1.reset_index()
all_enc_5=pd.merge(c,dups_per1,how="outer")
print all_enc_5.shape
all_enc_5.head()

(16087, 14)


,Person_Nbr,index,Enc_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,ICD,Num_Encs,Duplicate_Exists
0,327.0,16.0,14361139.0,2011-12-04 11:00:00,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,"367.4,365.01,365.01,367.4,367.4",5.0,False
1,335.0,20.0,11783809.0,2013-10-16 17:45:00,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,"250.00,250.00",2.0,False
2,1515.0,132.0,6074701.0,2012-10-01 15:30:00,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,"250.00,373.00,367.4,250.00,373.00",5.0,False
3,1519.0,133.0,8583237.0,2014-07-01 17:45:00,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,"367.0,367.4,362.05,362.57,362.57,362.05,367.4",7.0,False
4,1628.0,156.0,9342972.0,2016-04-26 17:00:00,Never smoker,Medicaid HFS,0.0,0.00,0.0,135.0,71.0,"H04.123,E11.9,H52.223",3.0,False


In [ ]:
all_enc_5.to_csv('all_enc_5.csv')

In [25]:
for index,row in all_enc_5.iterrows():
    if row["Duplicate_Exists"]==True:
        print 'True found'

Adding Macula Readings

In [26]:
macula=pd.read_pickle(path+'macula_findings_for_Enc.pickle')

In [29]:
print macula.shape
print len(macula.Enc_Nbr.drop_duplicates())
print len(macula.Person_Nbr.drop_duplicates())
macula.head()

(80946, 29)
80946
16087


,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_Macula1_Finding,OD_Macula1_Modifer,OD_Macula1_Location,OD_Macula2_Finding,OD_Macula2_Modifer,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
74595,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54456,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54457,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat (-)DME,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54458,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54459,71d10fa4-3b0d-fa52-cfbd-ffd0f3d3532d,14968450,2014-05-25 10:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
macula=macula.reset_index()

In [100]:
m1=macula.dropna(subset=["OD_Macula1_Finding"])
print m1.shape
m1.head()

(39940, 30)


,index,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_Macula1_Finding,OD_Macula1_Modifer,OD_Macula1_Location,OD_Macula2_Finding,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
0,74595,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54456,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,54457,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat (-)DME,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,54458,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,54463,831d44f9-4ab7-8df8-8757-bb1eebd14e6b,10658304,2016-10-22 09:15:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
a=m1.loc[m1.groupby(['Person_Nbr'])['Enc_Date'].idxmax()]
print a.shape
a.head()

(15409, 30)


,index,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_Macula1_Finding,OD_Macula1_Modifer,OD_Macula1_Location,OD_Macula2_Finding,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
0,74595,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,54463,831d44f9-4ab7-8df8-8757-bb1eebd14e6b,10658304,2016-10-22 09:15:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,44553,63ecc259-208f-7ef2-b0f8-acf25e23c264,5786173,2016-04-05 10:15:00,ca793eec-f101-2505-42e6-57fefc6baf14,146,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,81657,918e81a9-5ea3-933a-6c73-219c07ffb733,14534672,2016-10-05 00:45:00,3afb193c-dbf7-9f64-5f95-447f81cd0bdf,196,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,79224,2bad0952-05f8-acc5-35f2-3c7ea950943d,3610887,2016-01-19 07:45:00,8101be56-67f1-cc74-49d0-9eb3f721cdc6,331,Large ME,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Add SL_Lens Readings

In [102]:
sl_lens=pd.read_pickle(path+'SL_Lens_for_Enc.pickle')
print sl_lens.shape
print len(sl_lens.Enc_Nbr.drop_duplicates())
print len(sl_lens.Person_Nbr.drop_duplicates())
sl_lens.head()

(80946, 7)
80946
16087


,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_SL_LENS,OS_SL_LENS
74679,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC
54483,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+ with trace ACC changes
54484,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+
54485,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+
54486,71d10fa4-3b0d-fa52-cfbd-ffd0f3d3532d,14968450,2014-05-25 10:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,NaN,NaN


In [103]:
sl_lens.to_csv('sl_lens.csv')

In [104]:
sl_lens=sl_lens.reset_index()

In [105]:
sl=sl_lens.dropna(subset=["OD_SL_LENS"])
print sl.shape
sl.head()

(54383, 8)


,index,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_SL_LENS,OS_SL_LENS
0,74679,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC
1,54483,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+ with trace ACC changes
2,54484,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+
3,54485,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+
5,54487,6073586c-f74c-dda4-1a77-7ae779034993,14537805,2015-03-14 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,clear lens capsule cortex and nucleus,clear lens capsule cortex and nucleus


In [106]:
b=sl.loc[sl.groupby(['Person_Nbr'])['Enc_Date'].idxmax()]
print b.shape
b.head()

(15783, 8)


,index,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_SL_LENS,OS_SL_LENS
0,74679,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC
8,54490,831d44f9-4ab7-8df8-8757-bb1eebd14e6b,10658304,2016-10-22 09:15:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,clear lens capsule cortex and nucleus,clear lens capsule cortex and nucleus
14,44579,63ecc259-208f-7ef2-b0f8-acf25e23c264,5786173,2016-04-05 10:15:00,ca793eec-f101-2505-42e6-57fefc6baf14,146,2 + NS (undilated),2 + NS (undilated)
15,81629,918e81a9-5ea3-933a-6c73-219c07ffb733,14534672,2016-10-05 00:45:00,3afb193c-dbf7-9f64-5f95-447f81cd0bdf,196,1+ NS,1+ NS
20,79265,2bad0952-05f8-acc5-35f2-3c7ea950943d,3610887,2016-01-19 07:45:00,8101be56-67f1-cc74-49d0-9eb3f721cdc6,331,nuclear sclerosis 1+,nuclear sclerosis 1+


In [107]:
all_enc_5.shape

(16087, 14)

In [108]:
all_enc_6=pd.merge(all_enc_5,a, how='left',on='Person_Nbr')
print all_enc_6.shape
all_enc_6.head()

(16087, 43)


,Person_Nbr,index_x,Enc_Nbr_x,Enc_Date_x,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
0,327.0,16.0,14361139.0,2011-12-04 11:00:00,Never smoker,Medicare,0.0,0.00,121.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,335.0,20.0,11783809.0,2013-10-16 17:45:00,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,...,NaN,no CSME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1515.0,132.0,6074701.0,2012-10-01 15:30:00,Never smoker,VOHHA,0.0,0.00,200.0,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1519.0,133.0,8583237.0,2014-07-01 17:45:00,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,...,NaN,(-) CSME,normal,NaN,(+) scattered hard drusen (+) macula/post pole,NaN,NaN,NaN,NaN,NaN
4,1628.0,156.0,9342972.0,2016-04-26 17:00:00,Never smoker,Medicaid HFS,0.0,0.00,0.0,135.0,...,NaN,(-)CSME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
all_enc_6=pd.merge(all_enc_6,b, how='left',on='Person_Nbr')
print all_enc_6.shape
all_enc_6.head()

(16087, 50)


,Person_Nbr,index_x,Enc_Nbr_x,Enc_Date_x,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,...,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location,index,Enc_ID_y,Enc_Nbr,Enc_Date,Person_ID_y,OD_SL_LENS,OS_SL_LENS
0,327.0,16.0,14361139.0,2011-12-04 11:00:00,Never smoker,Medicare,0.0,0.00,121.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,335.0,20.0,11783809.0,2013-10-16 17:45:00,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,...,NaN,NaN,NaN,67224.0,951dc7d9-b75a-0193-6e3e-5cacf9236552,11783809.0,2013-10-16 17:45:00,2852c329-0938-7863-2a4c-38c9940a815a,tr NS,tr NS
2,1515.0,132.0,6074701.0,2012-10-01 15:30:00,Never smoker,VOHHA,0.0,0.00,200.0,130.0,...,NaN,NaN,NaN,14706.0,6ab5bf81-2df1-1e9a-c391-b67f3eecbcf0,6074701.0,2012-10-01 15:30:00,d0d5f2ca-3810-b543-80c6-3de3394eb7b1,nuclear sclerosis 1+,nuclear sclerosis 1+
3,1519.0,133.0,8583237.0,2014-07-01 17:45:00,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,...,NaN,NaN,NaN,71687.0,a78cf3fd-d47d-3e90-e0f5-c18a5d93fa7d,8583237.0,2014-07-01 17:45:00,fad736b3-9005-2553-644e-47b1a79152f8,trace nuclear,trace nuclear
4,1628.0,156.0,9342972.0,2016-04-26 17:00:00,Never smoker,Medicaid HFS,0.0,0.00,0.0,135.0,...,NaN,NaN,NaN,34980.0,f9cc6bba-4ceb-fc68-e480-e9c54755765f,9342972.0,2016-04-26 17:00:00,06b6ffb3-0944-117d-cd37-df0e38e89ab4,clear lens capsule cortex and nucleus,clear lens capsule cortex and nucleus


In [111]:
all_enc_6.to_csv("all_enc_6.csv")

Adding family hist

In [112]:
family_hist=pd.read_pickle(path+'family_hist_list.pickle')
print family_hist.shape
print len(family_hist.Person_Nbr.drop_duplicates())
family_hist.head()

(59203, 7)
14044


,Person_ID,Person_Nbr,Date_Created,Code,Code_System,Family_History,Relation
47872,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:15.980,160347007,SNOMED,Glaucoma,Father
47869,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:26.897,160274005,SNOMED,No history of Diabetes mellitus,Mother
47867,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:28.590,160274005,SNOMED,No history of Diabetes mellitus,Father
47871,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:29.563,160267000,SNOMED,No history of Glaucoma,Mother
47868,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 15:55:35.837,439724007,SNOMED,Heart disease,Brother


In [113]:
family_hist=family_hist.reset_index()

In [150]:
c=family_hist.groupby(["Person_Nbr","Relation"])

In [151]:
print len(c)
c.head()

31851


,index,Person_ID,Person_Nbr,Date_Created,Code,Code_System,Family_History,Relation
0,47872,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:15.980,160347007,SNOMED,Glaucoma,Father
1,47869,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:26.897,160274005,SNOMED,No history of Diabetes mellitus,Mother
2,47867,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:28.590,160274005,SNOMED,No history of Diabetes mellitus,Father
3,47871,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:29.563,160267000,SNOMED,No history of Glaucoma,Mother
4,47868,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 15:55:35.837,439724007,SNOMED,Heart disease,Brother
5,47870,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 15:55:47.830,439724007,SNOMED,Heart disease,Sister
6,31261,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,2015-07-14 11:37:20.683,416855002,SNOMED,Diabetes mellitus,Mother
7,31260,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,2015-07-14 11:37:22.413,160274005,SNOMED,No history of Diabetes mellitus,Father
8,31262,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,2015-07-14 11:37:28.213,160267000,SNOMED,No history of Glaucoma,Mother
9,31259,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,2015-07-14 11:37:29.150,160267000,SNOMED,No history of Glaucoma,Father


In [169]:
family_hist.to_csv('family_hist.csv')

In [ ]:
fam_his=pd.read_csv(interpath+'family_hist-csv (1).csv')
print fam_his.shape
print len(fam_his.Person_Nbr.drop_duplicates())
fam_his.head()

In [ ]:
fam_his1=fam_his.dropna(subset=["Code"])
print fam_his1.shape
print len(fam_his1.Person_Nbr.drop_duplicates())
fam_his1.head()

In [ ]:
print len(fam_his1.Relation.unique())
fam_his1.Relation.unique()

In [ ]:
cols=["Person_Nbr",'Relation',"Code"]
b=fam_his1[cols]
b.head()

In [ ]:
count=0
index=0
e=pd.DataFrame(columns=['Person_Nbr', 'Relation',"Code"])
for name, group in b.groupby(['Person_Nbr', 'Relation']):
    print(name)
    print len(group.get_values())
    if len(e)==0:
        e.loc[index,"Person_Nbr"]=name[0]
        e.loc[index,"Relation"]=(name[1])
        for i in range(len(group.get_values())):
            print "in for:"+ str(i)
            e.loc[index,"Code"]=str(e.loc[index,"Code"])+','+str(group.get_value(index=i,col=2,takeable=True))
        e.loc[index,"Code"]=e.loc[index,"Code"]+';'
        index=index+1
    elif (e.loc[index-1,"Person_Nbr"]==name[0] ):
        e.loc[index-1,"Person_Nbr"]=name[0]
        e.loc[index-1,"Relation"]=(name[1])
        for i in range(len(group.get_values())):
            print "in for:"+ str(i)
            e.loc[index-1,"Code"]=str(e.loc[index-1,"Code"])+','+str(group.get_value(index=i,col=2,takeable=True))
        e.loc[index-1,"Code"]=e.loc[index-1,"Code"]+';'
    else:
        e.loc[index,"Person_Nbr"]=name[0]
        e.loc[index,"Relation"]=(name[1])
        for i in range(len(group.get_values())):
            print "in for:"+ str(i)
            e.loc[index,"Code"]=str(e.loc[index,"Code"])+','+str(group.get_value(index=i,col=2,takeable=True))
        e.loc[index,"Code"]=e.loc[index,"Code"]+';'
        index=index+1
    count=count+1
    if count==4:
        break

In [284]:
diagnoses = {
            # Diabetes is under 250.* and 362.0.* for ICD9 and E08,E09,E10,E11,E13,O24 for ICD10
            "DM" : "250.*|362\.0.*|E(?:0[89]|1[013])(?:\.[A-Z0-9]{1,4})?|O24.*",
            # Macular edema is under 362.07 for ICD9 and E(08|09|10|11|13).3([1-5]1|7) for ICD10
            "ME" : "362\.07|E(?:0[89]|1[013])\.3(?:[1-5]1|7).*",
            # Mild Nonproliferative Diabetic Retinopathy is under 362.04 for ICD9 and E(08|09|10|11|13).32
            "mNPDR" : "362\.04|E(?:0[89]|1[013])\.32.*",
            # Moderate Nonproliferative Diabetic Retinopathy is under 362.05 for ICD9 and E(08|09|10|11|13).33
            "MNPDR" : "362\.05|E(?:0[89]|1[013])\.33.*",
            # Severe Nonproliferative Diabetic Retinopathy is under 362.06 for ICD9 and E(08|09|10|11|13).34
            "SNPDR" : "362\.06|E(?:0[89]|1[013])\.34.*",
            # Proliferative Diabetic Retinopathy is under 362.02 for ICD9 and E(08|09|10|11|13).35
            "PDR" : "362\.02|E(?:0[89]|1[013])\.35.*",
            # Glaucoma Suspect is under 365.0 for ICD9 and H40.0 for ICD10
            "Glaucoma_Suspect" : "365\.0.*|H40\.0.*",
            # Open-angle Glaucoma is under 365.1 for ICD9 and H40.1 for ICD10
            "Open_angle_Glaucoma" : "365\.1.*|H40\.1.*",
            # Cataract is under 366 for ICD9 and H25 and H26 for ICD10
            "Cataract" : "366(?:\.\d{1,2})?|H2[56](?:\.[A-Z0-9]{1,4})?"
        }

In [285]:
for diagnosis, pattern in diagnoses.iteritems():
        all_enc_4[diagnosis]=all_enc_4["ICD"].str.contains(pattern)